In [23]:
import keras.losses
import tensorflow as tf
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np
from keras.layers import Dense, BatchNormalization, Dropout
from keras.models import Sequential
from sklearn.utils import class_weight

In [24]:
model = Xception(weights="imagenet", include_top=True)

In [25]:
def get_features(ID):
    img_path = 'food_porn/food/' + ID + '.jpg'
    img = image.load_img(img_path, target_size=(299, 299))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    features = np.array(model.predict(x))
    return features

In [26]:
data = np.zeros((10000,1001))
id = np.array(range(0,10000))
data[:,0] = id.transpose()

def features():
    for i in range(0,10000):
        featureid = '{0:05}'.format(i)
        features = get_features(featureid)
        data[i,1:] = features
        if i % 100 == True:
            print(i)
    return data

using false we got:     527 correct
using true we got:      520 correct

how about... we use the true version to feed a NN

In [27]:
train_trip = np.loadtxt("train_triplets.txt").astype('int')

data = np.genfromtxt("features_fully_connected.csv", delimiter= ',')
data = np.array(data[:,1:])

def load_features(int_id):
    return data[int_id]

In [28]:
%%time

#create the base dataset with the 2 sets of labels
data_y1 = np.ones((59515,1))
data_y0 = np.zeros((59515,1))

#create the inverse dataset:
train_trip_inv = np.zeros((59515,3)).astype('int')
train_trip_inv[:,0] = train_trip[:,0]
train_trip_inv[:,1] = train_trip[:,2]
train_trip_inv[:,2] = train_trip[:,1]

#assemble everything into one huge pile of S*** in order to shuffle it
id_fin = np.concatenate((train_trip,train_trip_inv),axis=0)
labels = np.concatenate((data_y1,data_y0),axis=0)
id_labels = np.concatenate((id_fin,labels),axis=1)

data_nn_x = np.zeros((119030,3000))

for i in range(0,3):
        for j in range(0,119030):
            data_nn_x[j,range(i*1000,1000*(i+1))] = load_features(id_labels[j,i].astype('int'))
            if j%10000==False:
                print("we're at entry:", i, j)

test_trip = np.loadtxt("test_triplets.txt").astype('int')
test_nn_x = np.zeros((119088,3000))
for i in range(0,3):
        for j in range(0,119088):
            test_nn_x[j,range(i*1000,1000*(i+1))] = load_features(id_labels[j,i].astype('int'))
            if j%10000==False:
                print("we're at entry:", i, j)


#print(train_trip)
#print(train_trip_inv)
#now we try to randomize the data in order to increase our accuracy
# we may even double the dataset by using 0 as well as the 1 case

we're at entry: 0 0
we're at entry: 0 10000
we're at entry: 0 20000
we're at entry: 0 30000
we're at entry: 0 40000
we're at entry: 0 50000
we're at entry: 0 60000
we're at entry: 0 70000
we're at entry: 0 80000
we're at entry: 0 90000
we're at entry: 0 100000
we're at entry: 0 110000
we're at entry: 1 0
we're at entry: 1 10000
we're at entry: 1 20000
we're at entry: 1 30000
we're at entry: 1 40000
we're at entry: 1 50000
we're at entry: 1 60000
we're at entry: 1 70000
we're at entry: 1 80000
we're at entry: 1 90000
we're at entry: 1 100000
we're at entry: 1 110000
we're at entry: 2 0
we're at entry: 2 10000
we're at entry: 2 20000
we're at entry: 2 30000
we're at entry: 2 40000
we're at entry: 2 50000
we're at entry: 2 60000
we're at entry: 2 70000
we're at entry: 2 80000
we're at entry: 2 90000
we're at entry: 2 100000
we're at entry: 2 110000
Wall time: 39.2 s


In [29]:
print("%d bytes" % (data_nn_x.size * data_nn_x.itemsize))
#np.savetxt("nn_data.csv",data_nn_x,delimiter=',')

2856720000 bytes


In [36]:
#TAKE IT Branda:
neuralNetwork = Sequential()
neuralNetwork.add(Dense(128, activation='relu', input_dim=3000))
neuralNetwork.add(BatchNormalization())
neuralNetwork.add(Dense(128, activation='relu'))
neuralNetwork.add(BatchNormalization())
neuralNetwork.add(Dropout(0.2))
neuralNetwork.add(Dense(128, activation='relu'))
neuralNetwork.add(BatchNormalization())
neuralNetwork.add(Dropout(0.2))
neuralNetwork.add(Dense(128, activation='relu'))
neuralNetwork.add(BatchNormalization())
neuralNetwork.add(Dropout(0.2))
neuralNetwork.add(Dense(128, activation='relu'))
neuralNetwork.add(BatchNormalization())
neuralNetwork.add(Dropout(0.2))
neuralNetwork.add(Dense(128, activation='relu'))
neuralNetwork.add(BatchNormalization())
neuralNetwork.add(Dropout(0.2))
neuralNetwork.add(Dense(128, activation='relu'))
neuralNetwork.add(BatchNormalization())
neuralNetwork.add(Dropout(0.2))
neuralNetwork.add(Dense(1, activation='sigmoid'))
#neuralNetwork.add(BatchNormalization())

neuralNetwork.compile(loss = keras.losses.BinaryCrossentropy(), optimizer = 'adam', metrics=['accuracy'])

#fit the network to (for now the un-sparse matrix)
neuralNetwork.fit(data_nn_x, labels, epochs=3000, batch_size=1024, verbose=2, validation_split = 0.2)

#evaluation of the network prediction
#predict_test_nn = neuralNetwork.predict(df_test_onehot_nn)
#predict_train_nn = neuralNetwork.predict(df_train_onehot_nn)
#print(predict_test_nn)
#print(predict_train_nn)
#predict_test_nn = (predict_test_nn >= 0.5)
neuralNetwork.summary()

Epoch 1/3000
93/93 - 1s - loss: 0.7372 - accuracy: 0.5511 - val_loss: 0.9669 - val_accuracy: 0.0000e+00
Epoch 2/3000
93/93 - 1s - loss: 0.6845 - accuracy: 0.5966 - val_loss: 0.9673 - val_accuracy: 0.0000e+00
Epoch 3/3000
93/93 - 1s - loss: 0.6733 - accuracy: 0.6099 - val_loss: 0.9260 - val_accuracy: 0.0000e+00
Epoch 4/3000
93/93 - 1s - loss: 0.6677 - accuracy: 0.6180 - val_loss: 0.9337 - val_accuracy: 0.0000e+00
Epoch 5/3000
93/93 - 1s - loss: 0.6649 - accuracy: 0.6209 - val_loss: 0.9372 - val_accuracy: 0.0000e+00
Epoch 6/3000
93/93 - 1s - loss: 0.6635 - accuracy: 0.6230 - val_loss: 0.9797 - val_accuracy: 0.0000e+00
Epoch 7/3000
93/93 - 1s - loss: 0.6620 - accuracy: 0.6236 - val_loss: 0.9781 - val_accuracy: 0.0000e+00
Epoch 8/3000
93/93 - 1s - loss: 0.6619 - accuracy: 0.6244 - val_loss: 0.9692 - val_accuracy: 0.0000e+00
Epoch 9/3000
93/93 - 1s - loss: 0.6605 - accuracy: 0.6246 - val_loss: 0.9845 - val_accuracy: 3.3605e-04
Epoch 10/3000
93/93 - 1s - loss: 0.6596 - accuracy: 0.6249 - val

In [42]:
test_trip = np.loadtxt("test_triplets.txt").astype('int')
test_nn_x = np.zeros((59544,3000))
for i in range(0,3):
        for j in range(0,59544):
            test_nn_x[j,range(i*1000,1000*(i+1))] = load_features(test_trip[j,i].astype('int'))
            if j%10000==False:
                print("we're at entry:", i, j)

predict_test_nn = neuralNetwork.predict(test_nn_x)

we're at entry: 0 0
we're at entry: 0 10000
we're at entry: 0 20000
we're at entry: 0 30000
we're at entry: 0 40000
we're at entry: 0 50000
we're at entry: 1 0
we're at entry: 1 10000
we're at entry: 1 20000
we're at entry: 1 30000
we're at entry: 1 40000
we're at entry: 1 50000
we're at entry: 2 0
we're at entry: 2 10000
we're at entry: 2 20000
we're at entry: 2 30000
we're at entry: 2 40000
we're at entry: 2 50000


In [47]:
export = np.array(predict_test_nn>0.5).astype('int')
np.savetxt("submission.txt",np.round(export,decimals=0), fmt='%i')